In [24]:
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

import tensorflow.keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array


DATA_PATH = '~/data/isic-2024-challenge'

In [25]:
from PIL import Image
print(f"Pillow version: {Image.__version__}")

Pillow version: 10.4.0


In [26]:
for _, _, filenames in os.walk(f'{DATA_PATH}/train-image/image'):
    all_images = filenames

In [27]:
image_id = "4049849"

image_name =f"{DATA_PATH}/train-image/image/ISIC_{image_id}.jpg"

print(image_name)

image = load_img(image_name, color_mode='grayscale')

image_array = img_to_array(image) / 255.0

image_resized = tf.image.resize(image_array, (32, 32))

image_numpy = image_resized.numpy()

flattened_image = image_numpy.flatten()

print(min(flattened_image))
print(max(flattened_image))

~/data/isic-2024-challenge/train-image/image/ISIC_4049849.jpg


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [28]:
df = pd.read_csv(
    f"{DATA_PATH}/train-metadata.csv",
    dtype = {
        'isic_id': 'string',
        'target': 'int64',
        'patient_id': 'string',
        'age_approx': 'float64',
        'sex': 'string',
        'anatom_site_general': 'string',
        'clin_size_long_diam_mm': 'float64',
        'image_type': 'string',
        'tbp_tile_type': 'string',
        'tbp_lv_A': 'float64',
        'tbp_lv_Aext': 'float64',
        'tbp_lv_B': 'float64',
        'tbp_lv_Bext': 'float64',
        'tbp_lv_C': 'float64',
        'tbp_lv_Cext': 'float64',
        'tbp_lv_H': 'float64',
        'tbp_lv_Hext': 'float64',
        'tbp_lv_L': 'float64',
        'tbp_lv_Lext': 'float64',
        'tbp_lv_areaMM2': 'float64',
        'tbp_lv_area_perim_ratio': 'float64',
        'tbp_lv_color_std_mean': 'float64',
        'tbp_lv_deltaA': 'float64',
        'tbp_lv_deltaB': 'float64',
        'tbp_lv_deltaL': 'float64',
        'tbp_lv_deltaLB': 'float64',
        'tbp_lv_deltaLBnorm': 'float64',
        'tbp_lv_eccentricity': 'float64',
        'tbp_lv_location': 'string',
        'tbp_lv_location_simple': 'string',
        'tbp_lv_minorAxisMM': 'float64',
        'tbp_lv_nevi_confidence': 'float64',
        'tbp_lv_norm_border': 'float64',
        'tbp_lv_norm_color': 'float64',
        'tbp_lv_perimeterMM': 'float64',
        'tbp_lv_radial_color_std_max': 'float64',
        'tbp_lv_stdL': 'float64',
        'tbp_lv_stdLExt': 'float64',
        'tbp_lv_symm_2axis': 'float64',
        'tbp_lv_symm_2axis_angle': 'int64',
        'tbp_lv_x': 'float64',
        'tbp_lv_y': 'float64',
        'tbp_lv_z': 'float64',
        'attribution': 'string',
        'copyright_license': 'string',
        'lesion_id': 'string',
        'iddx_full': 'string',
        'iddx_1': 'string',
        'iddx_2': 'string',
        'iddx_3': 'string',
        'iddx_4': 'string',
        'iddx_5': 'string',
        'mel_mitotic_index': 'string',
        'mel_thick_mm': 'float64',
        'tbp_lv_dnn_lesion_confidence': 'float64'
    }
)

In [29]:
df["target"].value_counts()

target
0    400666
1       393
Name: count, dtype: int64